# Güneş Paneli Akım Geririlim Hesabı
Projemizin amacı, PAN dosyasından çıkarılan verilerle bir güneş panelinin belirli bir anın ışınım ve sıcaklık değerlerine göre panelin akım ve gerilim değerlerini hesaplamaktır.

I = Isc + α(G-Gref)
V = Voc - β(Tc-Tref)


# PAN Dosyasının Okunması

In [81]:
def read_pan_file(filepath):
    """PAN dosyası okunur."""
    with open(filepath, 'r') as file:
        content = file.read()
    return content

# Güneş Paneli Veri İşleme
`SolarPanel` sınıfı, pan verilerini içerir ve veri parsing, parametre çıkartma gibi metodlara sahiptir. 
- `_parse_pan_data` metodu, PAN verisini key-value çiftleri olarak parse eder.
- `_extract_parameters` metodu, gereken parametreleri çıkartır ve bunları bir sözlük olarak döndürür.

In [82]:
class SolarPanel:
    def __init__(self, pan_data):
        self.raw_data = pan_data
        self.parsed_data = self._parse_pan_data()
        self.parameters = self._extract_parameters()
    
    def _parse_pan_data(self):
        """Veri çiftlerini ayrıştırır ve anahtar-değer çiftleri olarak döndürür."""
        data = {}
        for line in self.raw_data.split("\n"):
            parts = line.split("=")
            if len(parts) == 2:
                key, value = parts
                data[key.strip()] = value.strip()
        return data
    
    def _extract_parameters(self):
        """Gerekli parametreleri çıkarır ve bir sözlük olarak döndürür.
        
        Çıkarılan parametreler:
        - 'alpha_sc': Kısa devre akımı sıcaklık katsayısı (A/°C)
        - 'I_L_ref': Referans koşullarında ışıkla oluşturulan akım (A)
        - 'V_oc_ref': Referans koşullarında açık devre gerilimi (V)
        - 'beta': Açık devre gerilimi sıcaklık katsayısı (V/°C)
        """
        params = {
            'alpha_sc': float(self.parsed_data.get('muISC', 0)) / 100, # muISC %/°C birimindedir, A/°C'ye dönüştürülür
            'I_L_ref': float(self.parsed_data.get('Isc', 0)), # Isc A birimindedir
            'V_oc_ref': float(self.parsed_data.get('Voc', 0)), # Voc V birimindedir
            'beta': float(self.parsed_data.get('muVocSpec',0)) / 1000 # muVocSpec mV/°C birimindedir, V/°C'ye dönüştürülür
        }
        return params
    
    def calculate_current(self, irradiance):
        """Radyans bağlı olarak akımı hesaplar (A).
        
        Akım hesaplama formülü:
        \[ I = Isc + \alpha \times (G - G_{ref}) \]
        
        Burada:
        - \(I\) : Çıkış akımı (A)
        - \(Isc\) : Kısa devre akımı (A)
        - \(\alpha\) : Kısa devre akımı sıcaklık katsayısı (A/°C)
        - \(G\) : Radyans (W/m^2)
        - \(G_{ref}\) : Referans radyans, genellikle 1000 W/m^2
        """
        I_L = self.parameters['I_L_ref'] + self.parameters['alpha_sc'] * (irradiance - 1000)
        return I_L
    
    def calculate_voltage(self, temperature):
        """Hücre sıcaklığına bağlı olarak gerilimi hesaplar (V).
        
        Gerilim hesaplama formülü:
        \[ V = Voc - \beta \times (Tc - T_{ref}) \]
        
        Burada:
        - \(V\) : Çıkış gerilimi (V)
        - \(Voc\) : Açık devre gerilimi (V)
        - \(\beta\) : Açık devre gerilimi sıcaklık katsayısı (V/°C)
        - \(Tc\) : Hücre sıcaklığı (°C)
        - \(T_{ref}\) : Referans sıcaklık, genellikle 25 °C
        """
        V_oc = self.parameters['V_oc_ref'] - (self.parameters['beta'] * (temperature - 25))  
        return V_oc


## Veri Okuma
Örnek bir PAN dosyası içeriği read_pan_file fonksiyonu ile okunur ve SolarPanel sınıfının parametreleri ekrana yazdırılır.

In [83]:
pan_file_content = read_pan_file('C:\\Users\\Pratikus\\Documents\\GitHub\\SolarEnergyPredictionSuite\\data_acquisition\\equipment_specs\\module_specs\\ALFA_370.PAN')
solar_panel = SolarPanel(pan_file_content)
print(solar_panel.parameters)

{'alpha_sc': 0.05, 'I_L_ref': 9.84, 'V_oc_ref': 48.0, 'beta': -0.14}


# Sonuçların Hesaplanması ve Gösterimi
Örnek ışınım ve sıcaklık değerleri ile akım ve gerilim hesaplamaları yapılır.

In [84]:
pan_file_content

'PVObject_=pvModule\n  Version=6.79\n  Flags=$0303\n\n  PVObject_Commercial=pvCommercial\n    Flags=$0041\n    Manufacturer=ALFASOLAR\n    Model=ALFA_370\n    DataSource=ALFOSOLAR\n    Width=0.997\n    Height=1.969\n    NPieces=100\n    PriceDate=28/05/19 17:52\n  End of PVObject pvCommercial\n\n  Technol=mtSiMono\n  NCelS=72\n  NCelP=1\n  NDiode=1\n  GRef=1000\n  TRef=25.0\n  PNom=370.0\n  Isc=9.840\n  Voc=48.00\n  Imp=9.350\n  Vmp=40.00\n  muISC=5.00\n  muVocSpec=-140.0\n  muPmpReq=0.000\n  RShunt=400\n  Rp_0=2000\n  Rp_Exp=5.50\n  RSerie=0.245\n  Gamma=0.960\n  muGamma=0.0000\n  Absorb=0.90\n  ARev=3.200\n  BRev=10.356\n  RDiode=0.010\n  VRevDiode=-0.70\n  AirMassRef=1.500\n  SandiaAMCorr=50.000\nEnd of PVObject pvModule\n'

In [87]:
# Örnek ışınım ve sıcaklık değerleri
example_irradiance = 900  # W/m^2
example_temperature = 35  # C

# Akım ve Gerilim hesaplamaları
current = solar_panel.calculate_current(example_irradiance)
voltage = solar_panel.calculate_voltage(example_temperature)

# Sonuçların Ekrana Yazdırılması
print("{} W/m^2 ışınım altında ve {} °C hücre sıcaklığında, "
      "hesaplanan çıkış akımı {:.2f} A ve çıkış gerilimi {:.2f} V olarak bulunmuştur."
      .format(example_irradiance, example_temperature, current, voltage))

900 W/m^2 ışınım altında ve 35 °C hücre sıcaklığında, hesaplanan çıkış akımı 4.84 A ve çıkış gerilimi 49.40 V olarak bulunmuştur.
